In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim, nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.data import Dataset 
import torch.utils.data as utils_data
import IPython
import torchvision

training_set = os.listdir('../input/dogs-cats-images/dog vs cat/dataset/training_set/')#/dataset/training_set/')

testing_set = os.listdir('../input/dogs-cats-images/dog vs cat/dataset/test_set/')



In [ ]:
size = 100

ModifyPics = transforms.Compose([transforms.Resize((size,size)),
                                transforms.ToTensor(),
                                transforms.Normalize([.5, .5, .5],
                                                     [.5, .5, .5])])
train_on_gpu = torch.cuda.is_available()


if not train_on_gpu:
    print("Cuda is NOT Avaialble... cpu usage")
    
else:
    print("Cuda IS Avaialble... gpu usage")
    
torch.cuda.empty_cache()

device = torch.device("cuda:0")

train_set =datasets.ImageFolder(
    "../input/dogs-cats-images/dog vs cat/dataset/training_set/",
    transform=ModifyPics)


train_loader=torch.utils.data.DataLoader(
    train_set,batch_size=50, 
    shuffle=True)

test_set=datasets.ImageFolder(
    "../input/dogs-cats-images/dog vs cat/dataset/test_set/",
    transform=ModifyPics)
test_loader=torch.utils.data.DataLoader(
    test_set,batch_size=50,shuffle=True)




In [ ]:
%matplotlib inline 
batch = next(iter(train_loader))

images, labels = batch
print(labels)
# grid = torchvision.utils.make_grid(images, nrow=10)
# plt.figure(figsize=(50,50))
# plt.imshow(np.transpose(grid, (1,2,0)))


In [ ]:
def validation(model, validateloader, criterion):
    
    val_loss = 0
    accuracy = 0
    
    for images, labels in iter(validateloader):
        

        images,labels=images.to(device),labels.to(device)
        

        output = model.forward(images)
        val_loss += criterion(output, labels).item()

        probabilities = torch.exp(output)
        
        equality = (labels.data == probabilities.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return val_loss, accuracy

In [ ]:

import torch.nn.functional as F
device = torch.device("cuda:0")


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=128,kernel_size= 5,padding=1)
        self.conv2 = nn.Conv2d(in_channels=128, out_channels=64,kernel_size= 5,padding =1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size= 5, padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1  = nn.Linear(9248,256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64,2)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self,x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        x = x.view(-1,9248)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        x = self.fc4(x)
        
        return x
        
        
        
    

In [ ]:
model = Net()
if train_on_gpu:
    model.cuda()

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=3e-2)

epochs = 30
train_losses, test_losses,accuracy_losses=[],[],[]
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        validation_loss = 0
        accuracy= 0
        
        with torch.no_grad():
            validation_loss, accuracy = validation(model, train_loader, criterion)
        
        model.train()
        train_losses.append(running_loss/len(test_loader))
        test_losses.append(validation_loss/len(test_loader))
        accuracy_losses.append(accuracy/len(test_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
              "Validation Loss: {:.3f}.. ".format(validation_loss/len(train_loader)),
              "Validation Accuracy: {:.3f}%".format(accuracy/len(train_loader)* 100))

        
        

        
        

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.plot(accuracy_losses,label='Validation Accuracy')
plt.legend(frameon=False)



In [ ]:
%matplotlib inline
# import pygame 
from PIL import Image 
from time import sleep
import torchvision.transforms.functional as TF
file1 = "../input/sounds/dog-sound/i_see_dog.wav"


file2 = "../input/sounds/dog-sound/i_see_cat.wav"


for i in range(5):
    images, labels = next(iter(train_loader)) 
    images,labels=images.to(device),labels.to(device)
    output = model(images)
    
   

    image = images[1,:]

    image = image.view(3,size,-1) 
    image = image.cpu().numpy()
    image = image /2+.05
    plt.imshow(np.transpose(image, (1,2,0))) 
    target_value = output.data.max(1, keepdim=True)[1][1].item() 
   
    if target_value == 1: 
       
        dog=IPython.display.display(IPython.display.Audio(file1))
        
    else: 
        
        cat=IPython.display.display(IPython.display.Audio(file2))
        


    plt.show()

    

for i in range(5):
    images, labels = next(iter(test_loader)) 
    images,labels=images.to(device),labels.to(device)
    output = model(images)
    
    
   

    image = images[1,:]

    image = image.view(3,size,-1) 
    image = image.cpu().numpy()
    image = image /2+.05
    plt.imshow(np.transpose(image, (1,2,0))) 
    target_value = output.data.max(1, keepdim=True)[1][1].item() 
   
    if target_value == 1:
       
        
       
        dog=IPython.display.display(IPython.display.Audio(file1))
        
    else: 
       
        cat=IPython.display.display(IPython.display.Audio(file2))
        


    plt.show()
    